In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Atuona,38,PF,2021-07-20 23:03:29,75,-9.8000,-139.0333,77.40,26.11
1,1,Hvide Sande,72,DK,2021-07-20 23:03:29,78,56.0045,8.1294,60.01,21.00
2,2,Ushuaia,75,AR,2021-07-20 22:58:50,64,-54.8000,-68.3000,33.46,12.66
3,3,Nikolskoye,91,RU,2021-07-20 22:58:54,96,59.7035,30.7861,59.02,5.73
4,4,Sharjah,83,AE,2021-07-20 23:03:30,51,25.3573,55.4033,97.25,13.15


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Atuona,38,PF,2021-07-20 23:03:29,75,-9.8000,-139.0333,77.40,26.11
5,5,Lorengau,100,PG,2021-07-20 23:03:30,75,-2.0226,147.2712,81.95,2.15
7,7,Westport,1,US,2021-07-20 23:03:31,77,41.1415,-73.3579,85.01,1.99
9,9,Hilo,87,US,2021-07-20 23:02:03,83,19.7297,-155.0900,79.25,1.99
11,11,Bolshaya Martynovka,42,RU,2021-07-20 23:03:32,45,47.2710,41.6679,83.48,8.77
12,12,Albany,45,US,2021-07-20 23:03:33,79,42.6001,-73.9662,82.33,6.49
20,20,Praya,75,CV,2021-07-20 23:03:35,78,14.9215,-23.5087,77.54,8.05
22,22,Yanam,100,IN,2021-07-20 23:03:36,87,16.7333,82.2167,78.37,10.00
25,25,Virginia Beach,86,US,2021-07-20 22:59:23,62,36.8529,-75.9780,86.04,6.73
27,27,Domoni,16,KM,2021-07-20 23:03:38,78,-12.2569,44.5319,75.96,3.20


In [9]:
preferred_cities_df.count()

City_ID       211
City          211
Cloudiness    211
Country       211
Date          211
Humidity      211
Lat           211
Lng           211
Max Temp      211
Wind Speed    211
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Atuona,PF,77.40,-9.8000,-139.0333,
5,Lorengau,PG,81.95,-2.0226,147.2712,
7,Westport,US,85.01,41.1415,-73.3579,
9,Hilo,US,79.25,19.7297,-155.0900,
11,Bolshaya Martynovka,RU,83.48,47.2710,41.6679,
12,Albany,US,82.33,42.6001,-73.9662,
20,Praya,CV,77.54,14.9215,-23.5087,
22,Yanam,IN,78.37,16.7333,82.2167,
25,Virginia Beach,US,86.04,36.8529,-75.9780,
27,Domoni,KM,75.96,-12.2569,44.5319,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [13]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))